In [1]:
import pandas as pd

# Read the Parquet file
df = pd.read_parquet("full.parquet")

# View all columns and their data types
print(df.dtypes)

instance_id                               int32
cluster_size                            float64
user_id                                   int64
database_id                               int64
query_id                                  int64
arrival_timestamp                datetime64[us]
compile_duration_ms                     float64
queue_duration_ms                         int64
execution_duration_ms                     int64
feature_fingerprint                      object
was_aborted                               int32
was_cached                                int32
cache_source_query_id                   float64
query_type                               object
num_permanent_tables_accessed           float64
num_external_tables_accessed            float64
num_system_tables_accessed              float64
read_table_ids                           object
write_table_ids                          object
mbytes_scanned                          float64
mbytes_spilled                          

In [8]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("ParquetFileExplorer") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/22 17:04:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
# Load the Parquet file
df = spark.read.parquet("full.parquet")

In [10]:
df.show(20)

+-----------+------------+-------+-----------+--------+--------------------+-------------------+-----------------+---------------------+--------------------+-----------+----------+---------------------+----------+-----------------------------+----------------------------+--------------------------+--------------------+---------------+--------------+--------------+---------+---------+----------------+
|instance_id|cluster_size|user_id|database_id|query_id|   arrival_timestamp|compile_duration_ms|queue_duration_ms|execution_duration_ms| feature_fingerprint|was_aborted|was_cached|cache_source_query_id|query_type|num_permanent_tables_accessed|num_external_tables_accessed|num_system_tables_accessed|      read_table_ids|write_table_ids|mbytes_scanned|mbytes_spilled|num_joins|num_scans|num_aggregations|
+-----------+------------+-------+-----------+--------+--------------------+-------------------+-----------------+---------------------+--------------------+-----------+----------+------------

In [2]:
from pyspark.sql import SparkSession
from confluent_kafka import Producer
import json
from datetime import datetime
import time

# Function to convert row to JSON serializable format
def row_to_dict(row):
    row_dict = row.asDict()
    for key, value in row_dict.items():
        if isinstance(value, datetime):
            row_dict[key] = value.isoformat()
    return row_dict

# Callback function to check for successful delivery
def delivery_report(err, msg):
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

# Create Spark session
spark = SparkSession.builder \
    .appName("KafkaPublisher") \
    .getOrCreate()

# Load your dataset into a Spark DataFrame
df = spark.read.parquet("full.parquet")

# Confluent Kafka producer configuration
conf = {
    'bootstrap.servers': 'localhost:9092',  # Change to your Kafka broker address
    'client.id': 'sagnik-producer',
    'acks': 'all'  # Ensure the producer waits for an acknowledgment
}

# Initialize Kafka producer
producer = Producer(conf)

# Continuously stream data
while True:
    # Select the first 20 rows
    first_20_rows = df.limit(20).collect()

    # Publish each row to Kafka
    for row in first_20_rows:
        message = row_to_dict(row)
        message_str = json.dumps(message)  # Convert dictionary to JSON string
        print(f"Broadcasting message: {message_str}")
        # Send message to Kafka topic 'sagnik'
        producer.produce(
            'sagnik',  # Kafka topic
            value=message_str,  # Message content
            callback=delivery_report  # Delivery callback
        )
        producer.flush()  # Ensure the message is sent

    # Sleep for a while before sending the next batch
    time.sleep(5)
    break

# Close the producer (this will never be reached in this infinite loop)
producer.flush()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/23 17:54:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
%3|1737651279.436|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651280.439|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 1 identical error(s) suppressed)
%3|1737651281.441|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 0ms in state CONNECT)


Broadcasting message: {"instance_id": 0, "cluster_size": null, "user_id": 1, "database_id": 0, "query_id": 4241475, "arrival_timestamp": "2024-02-29T23:59:59.462678", "compile_duration_ms": 8265.0, "queue_duration_ms": 0, "execution_duration_ms": 8281, "feature_fingerprint": "42e3eae744df217d738077f11c29c7fe25d752f66229273a7eda9be6aa2c34d1", "was_aborted": 0, "was_cached": 0, "cache_source_query_id": null, "query_type": "select", "num_permanent_tables_accessed": 0.0, "num_external_tables_accessed": 0.0, "num_system_tables_accessed": 15.0, "read_table_ids": null, "write_table_ids": null, "mbytes_scanned": 0.0, "mbytes_spilled": 0.0, "num_joins": 22, "num_scans": 0, "num_aggregations": 23}


%3|1737651283.447|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651286.459|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 1ms in state CONNECT)
%3|1737651287.462|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651288.464|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651292.482|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 1ms in state CONNECT)
%3|1737651294.488|FAIL|

Message delivery failed: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
Broadcasting message: {"instance_id": 0, "cluster_size": null, "user_id": 1, "database_id": 0, "query_id": 3379841, "arrival_timestamp": "2024-03-01T00:00:00.503618", "compile_duration_ms": 8220.0, "queue_duration_ms": 0, "execution_duration_ms": 8235, "feature_fingerprint": "42e3eae744df217d738077f11c29c7fe25d752f66229273a7eda9be6aa2c34d1", "was_aborted": 0, "was_cached": 0, "cache_source_query_id": null, "query_type": "select", "num_permanent_tables_accessed": 0.0, "num_external_tables_accessed": 0.0, "num_system_tables_accessed": 15.0, "read_table_ids": null, "write_table_ids": null, "mbytes_scanned": 0.0, "mbytes_spilled": 175.0, "num_joins": 22, "num_scans": 0, "num_aggregations": 23}


%3|1737651584.540|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651585.542|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 1ms in state CONNECT)
%3|1737651586.547|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 1ms in state CONNECT)
%3|1737651589.560|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651590.565|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651591.567|FAIL|

Message delivery failed: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
Broadcasting message: {"instance_id": 0, "cluster_size": null, "user_id": 1, "database_id": 0, "query_id": 3352242, "arrival_timestamp": "2024-03-01T00:00:01.432177", "compile_duration_ms": 7940.0, "queue_duration_ms": 0, "execution_duration_ms": 7956, "feature_fingerprint": "42e3eae744df217d738077f11c29c7fe25d752f66229273a7eda9be6aa2c34d1", "was_aborted": 0, "was_cached": 0, "cache_source_query_id": null, "query_type": "select", "num_permanent_tables_accessed": 0.0, "num_external_tables_accessed": 0.0, "num_system_tables_accessed": 15.0, "read_table_ids": null, "write_table_ids": null, "mbytes_scanned": 0.0, "mbytes_spilled": 0.0, "num_joins": 22, "num_scans": 0, "num_aggregations": 23}


%3|1737651885.647|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 0ms in state CONNECT, 3 identical error(s) suppressed)
%3|1737651886.652|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651887.658|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 1ms in state CONNECT)
%3|1737651892.677|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651898.706|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 0ms in state CON

Message delivered to sagnik [0]
Broadcasting message: {"instance_id": 0, "cluster_size": null, "user_id": 1, "database_id": 0, "query_id": 4025600, "arrival_timestamp": "2024-03-01T00:00:05.502624", "compile_duration_ms": 8032.0, "queue_duration_ms": 0, "execution_duration_ms": 8053, "feature_fingerprint": "42e3eae744df217d738077f11c29c7fe25d752f66229273a7eda9be6aa2c34d1", "was_aborted": 0, "was_cached": 0, "cache_source_query_id": null, "query_type": "select", "num_permanent_tables_accessed": 0.0, "num_external_tables_accessed": 0.0, "num_system_tables_accessed": 15.0, "read_table_ids": null, "write_table_ids": null, "mbytes_scanned": 0.0, "mbytes_spilled": 176.0, "num_joins": 22, "num_scans": 0, "num_aggregations": 23}
Message delivered to sagnik [0]
Broadcasting message: {"instance_id": 0, "cluster_size": null, "user_id": 1, "database_id": 0, "query_id": 3572468, "arrival_timestamp": "2024-03-01T00:00:05.964802", "compile_duration_ms": 7158.0, "queue_duration_ms": 0, "execution_dura

0

%3|1737652175.747|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 2ms in state CONNECT)
%3|1737652176.749|FAIL|sagnik-producer#producer-2| [thrd:wlan-172-029-219-224.pool.utn.de:9092/0]: wlan-172-029-219-224.pool.utn.de:9092/0: Connect to ipv4#172.29.219.224:9092 failed: Connection refused (after 3ms in state CONNECT)
%3|1737652177.752|FAIL|sagnik-producer#producer-2| [thrd:wlan-172-029-219-224.pool.utn.de:9092/0]: wlan-172-029-219-224.pool.utn.de:9092/0: Connect to ipv4#172.29.219.224:9092 failed: Connection refused (after 1ms in state CONNECT, 1 identical error(s) suppressed)
%3|1737652179.445|FAIL|sagnik-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 1 identical error(s) suppressed)


In [1]:
from confluent_kafka import Consumer, KafkaError

# Kafka consumer configuration
conf = {
    'bootstrap.servers': 'localhost:9092',  # Change to your Kafka broker address
    'group.id': 'sagnik-consumer-group',  # Consumer group ID
    'auto.offset.reset': 'earliest',  # Start reading messages from the beginning if no offset is stored
}

# Initialize Kafka consumer
consumer = Consumer(conf)

# Subscribe to the Kafka topic 'sagnik'
consumer.subscribe(['sagnik'])

# Continuously consume messages
try:
    while True:
        msg = consumer.poll(1.0)  # Poll for messages with a timeout of 1 second

        if msg is None:
            # No message received, continue polling
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition reached, can ignore
                print(f"End of partition reached {msg.topic()} [{msg.partition()}]")
            else:
                # Error while consuming
                print(f"Error while consuming: {msg.error()}")
            continue

        # Successfully received a message
        message_value = msg.value().decode('utf-8')  # Decode the message value from bytes to string
        print(f"Received message: {message_value}")

except KeyboardInterrupt:
    # Exit gracefully on Ctrl+C
    print("Consumer closed by user")

finally:
    # Close the consumer
    consumer.close()

%3|1737651218.172|FAIL|rdkafka#consumer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 2ms in state CONNECT)
%3|1737651219.170|FAIL|rdkafka#consumer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651220.173|FAIL|rdkafka#consumer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651221.177|FAIL|rdkafka#consumer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1737651222.182|FAIL|rdkafka#consumer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 1ms in state CONNECT)
%3|1737651223.174|FAIL|rdkafka#consumer-1| [thrd:localhost:9092

Consumer closed by user


In [1]:
#Load Parquet to DuckDB
import duckdb
con = duckdb.connect("redset.duckdb")
# df = con.execute("SELECT * FROM 'full.parquet' LIMIT 10000").fetchdf()
# print(df)


con.execute("CREATE TABLE redset_org AS SELECT * FROM 'full.parquet'")
result = con.execute("SELECT * FROM redset_org LIMIT 20").fetchdf()
print(result)
con.close()

    instance_id  cluster_size  user_id  database_id  query_id  \
0             0           NaN        1            0   4241475   
1             0           NaN        1            0   3379841   
2             0           NaN        1            0   3352242   
3             0           NaN        1            0   4025600   
4             0           NaN        1            0   3572468   
5             0           NaN        1            0   4402278   
6             0           NaN        1            0   3799533   
7             0           NaN        2            0   3543217   
8             0           NaN        1            0   3870044   
9             0           NaN        2            0   4311766   
10            0           NaN        1            0   3569208   
11            0           NaN        2            0   2859999   
12            0           NaN        1            0   3092640   
13            0           NaN        1            0   3597791   
14            0          

In [2]:
import os
print(f"DuckDB file location: {os.path.abspath('redset.duckdb')}")

DuckDB file location: /Users/sagnikdas/GitHub/Data-Streaming-Pipeline/redset.duckdb
